In [ ]:
import numpy as np
import pandas as pd
import mne

import sys
sys.path.append('../')
from src import data

import matplotlib.pylab as plt
from nilearn import datasets

!date -I

In [ ]:
%ls {data.path('external/pytepfit/')}

In [ ]:
sim_stc = mne.read_source_estimate(data.path('external/pytepfit/stc_sim_evoked_new'))
real_stc = mne.read_source_estimate(data.path('external/pytepfit/all_avg_high-stc'))

In [ ]:
subjects_dir=data.path('external/pytepfit/')

PLOT_3D = False

surfer_kwargs = dict(
    subject='fsaverage',
    subjects_dir=subjects_dir,
    initial_time=0.1,
    smoothing_steps=6,
    show_traces='label',
)

if PLOT_3D:
    surfer_kwargs['views'] = "axial"
    surfer_kwargs['hemi'] = "both"
else:
    surfer_kwargs["backend"] = "matplotlib"
    surfer_kwargs['hemi'] = "lh"
    surfer_kwargs['views'] = 'lat'

brain = real_stc.plot(**surfer_kwargs)

In [ ]:
df = pd.read_csv(data.path('external/pytepfit/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv'))
df

In [ ]:
stim_weights = np.load('../data/external/pytepfit/stim_weights.npy')
stim_locatin = np.argmax(stim_weights)

plt.rcParams['figure.figsize'] = [10, 3]
plt.figure()
plt.plot(stim_weights)
plt.vlines(stim_locatin,0,1,colors='r')
plt.show()

In [ ]:
stim_roi_label = df.iloc[stim_locatin]
stim_roi_label

In [ ]:
src = mne.read_source_spaces(data.path('external/pytepfit/fsaverage/fsaverage-src'), verbose=False)
src.plot(subjects_dir=subjects_dir)

In [ ]:
labels_Schaefer = mne.read_labels_from_annot('fsaverage',
                                             parc='Schaefer2018_200Parcels_7Networks_order', 
                                             subjects_dir=subjects_dir, 
                                             verbose=False)

labels_Schaefer_relevant = labels_Schaefer[1:-1]
labels_Schaefer_relevant_names = [l.name[:-3] for l in labels_Schaefer_relevant]

labels_Schaefer_relevant_names[:10]

In [ ]:
real_label_ts = mne.extract_label_time_course(real_stc, labels_Schaefer_relevant, src, mode='mean_flip')
sim_label_ts = mne.extract_label_time_course(sim_stc, labels_Schaefer_relevant, src, mode='mean_flip')

In [ ]:
sim_label_ts.shape

In [ ]:
len(labels_Schaefer[1:-1])

In [ ]:
plt.plot(real_label_ts[1:-1,:].T);

In [ ]:
plt.plot(sim_label_ts[1:-1,:].T);

In [ ]:
plt.plot(real_label_ts[1:-1,900:1300].T);

In [ ]:
plt.plot(sim_label_ts[1:-1,900:1300].T);

In [ ]:
ts_labels = np.array([l.name for l in labels_Schaefer[1:-1]])

In [ ]:
stim_loc_ts_labels = None

for i,l in enumerate(ts_labels):
    if l == stim_roi_label["ROI Name"]+'-lh':
        stim_loc_ts_labels  = i

stim_loc_ts_labels

In [ ]:
np.savez(data.path('external/pytepfit/source_estimates/simulated.npz'), ts=sim_label_ts[1:-1,900:1300].T, labels=ts_labels, stim_location=stim_loc_ts_labels)

In [ ]:
np.savez(data.path('external/pytepfit/source_estimates/empirical.npz'), ts=real_label_ts[1:-1,900:1300].T, labels=ts_labels, stim_location=stim_loc_ts_labels)